In [2]:
import pandas as pd
from keras.src.models import Sequential
from keras.src.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout, Input, AveragePooling2D, GlobalAveragePooling2D
from keras.src.optimizers import Adam
from sklearn.model_selection import train_test_split
from keras.src.callbacks import EarlyStopping
import os
import matplotlib.pyplot as plt
import numpy as np
import cv2 as cv
import glob
from keras.src.applications.resnet import ResNet152

In [ ]:
def createDataset(path, desired_size = 256):
    images = []
    labels = []
    
    for label, label_path in enumerate(glob.glob(path + "/*")):
        for img_path in glob.glob(label_path + "/*"):
            img = cv.imread(img_path)
            img = cv.resize(img, (desired_size, desired_size))
            img = img.astype(np.float32) / 255.0  # Normalizzazione
            images.append(img)
            labels.append(label)
            cv.imshow('', img)
            break
        break
    return np.array(images), np.array(labels)

In [ ]:
images, labels = createDataset('data', 256)

In [ ]:
np.save('images2', images)

In [ ]:
np.save('labels', labels)

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(np.load('images2.npy'), np.load('labels.npy'), test_size=0.2, shuffle=True, random_state = 1, stratify=np.load('labels.npy'))
x_val, x_test, y_val, y_test = train_test_split(x_val, y_val, test_size=0.05, shuffle=True, random_state = 1, stratify=y_val)
print("split done")

In [ ]:
np.save('x_train_rgb_64px', x_train)
np.save('y_train_rgb_64px', y_train)
np.save('x_test_rgb_64px', x_test)
np.save('y_test_rgb_64px', y_test)
np.save('x_val_rgb_64px', x_val)
np.save('y_val_rgb_64px', y_val)

In [3]:
x_train = np.load('x_train_rgb.npy')
y_train = np.load('y_train_rgb.npy')
x_val = np.load('x_val_rgb.npy')
y_val = np.load('y_val_rgb.npy')

In [ ]:
x_test = np.load('x_test_rgb.npy')
y_test = np.load('y_test_rgb.npy')

In [4]:
data_path = 'data'
dirs = os.listdir(data_path)

In [ ]:
'''models = Sequential(
    layers = [
        Input(shape=(224, 224, 3), batch_size=32),
        Conv2D(16, (3, 3), activation='relu'), ## prima era 32
        MaxPooling2D((2, 2)),
        Conv2D(32, (3, 3), activation='relu'), ## prima era 64
        MaxPooling2D((2, 2)),
        Conv2D(32, (3, 3), activation='relu'), ## prima era 64
        Flatten(),
        Dense(64, activation='relu'),
        Dropout(0.3),
        Dense(128, activation='relu'), 
        Dense(len(dirs), activation='softmax')
    ],
    name='softmax_model'
)
print("network created")'''

In [5]:
base_model = ResNet152(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [6]:
models = Sequential(
    layers = [
        base_model,
        GlobalAveragePooling2D(),
        Dropout(0.5),
        Dense(512, activation='relu'),
        Dropout(0.3),
        Dense(256, activation='relu'),
        Dropout(0.3),
        Dense(128, activation='relu'),
        Dropout(0.2),
        Dense(len(dirs), activation='softmax')
    ]
)

In [ ]:
models = Sequential(
    layers = [
        Input(shape=(224, 224, 3), batch_size=32),
        Conv2D(16, (7, 7), activation='relu'),
        AveragePooling2D((2, 2)),
        Conv2D(32, (7, 7), activation='relu'),
        AveragePooling2D((2, 2)),
        Conv2D(32, (5, 5), activation='relu'),
        AveragePooling2D((2, 2)),
        Conv2D(256, (3, 3), activation='relu'),
        Flatten(),
        Dropout(0.3),
        Dense(512, activation='relu'),
        Dropout(0.3),
        Dense(len(dirs), activation='softmax')
    ],
    name='softmax_model'
)
print("network created")

In [ ]:
models = Sequential(
    layers = [
        Input(shape=(224, 224, 3), batch_size=32),
        Conv2D(16, (7, 7), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(32, (7, 7), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(64, (5, 5), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(256, (3, 3), activation='relu'),
        Flatten(),
        Dropout(0.3),
        Dense(512, activation='relu'),
        Dropout(0.3),
        Dense(len(dirs), activation='softmax')
    ],
    name='softmax_model'
)
print("network created")

In [7]:
# Compiling the model
models.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
print("model compiled")
# learning_rate dentro Adam()

model compiled


In [8]:
early_stopping = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

In [9]:
n_epoche = 100

: 

In [10]:
# Fitting the model
history = models.fit(x_train, y_train, epochs=n_epoche, validation_data=(x_val, y_val), batch_size=32, callbacks=[early_stopping])
print("model fitted")

Epoch 1/100


In [ ]:
models.summary()

In [ ]:
# saving the model in a file
models.save(filepath=os.path.join(f'{n_epoche}_epoche_soft_max_model.keras'))
print("model saved")

In [ ]:
# plotting the accuracy and the loss
xx = np.arange(1, 31, 1)
plt.subplot(2,1,1)
plt.xlabel('Epoche')
plt.ylabel('Loss')
plt.plot(xx, history['loss'], c='red')
plt.plot(xx, history['val_loss'], c='orange')
plt.subplot(2,2,1)
plt.plot(xx, history['accuracy'], c='green')
# plt.plot(xx, history['root_mean_squared_error'], c='red')
# plt.plot(xx, history['val_root_mean_squared_error'], c='orange')
# plt.xlabel('Epoche')
# plt.ylabel('RMSE')
plt.show()